## HW2
### INFO 3401

In [1]:
import numpy as np
import pandas as pd
import sqlite3
import os

## Getting started

Download the [Lahman's baseball database](http://www.seanlahman.com/baseball-archive/statistics/) from Canvas. 
- You will need to unzip the file and include it in the same directory as your notebook. 
- You will have to name your unzipped file `lahmansbaseballdb.sqlite` 
- If you are not quite sure what to do for this step, please consult the section on the file system in the INFO Paratechnical Handbook which has additional information. Then please post to Canvas for help!
- Note the database is current only to 2018. 

In [2]:
# here is how you connect to the Lahman database and 
# describe its tables

import sqlite3

 # pass a string pointing to the .sqlite file on your machine
con = sqlite3.connect("lahmansbaseballdb.sqlite")

# get the db name
db_name = pd.read_sql("PRAGMA database_list;", con)["name"][0]

In [96]:
# Let's list the first five tables in the database

list_tables = "SELECT * FROM {}.sqlite_master WHERE type='table';".format(db_name)

lahmans = pd.read_sql(list_tables , con=con)

lahmans[0:5]

,type,name,tbl_name,rootpage,sql
0,table,allstarfull,allstarfull,2,"CREATE TABLE ""allstarfull"" (\n\t""ID"" INTEGER N..."
1,table,appearances,appearances,151,"CREATE TABLE ""appearances"" (\n\t""ID"" INTEGER N..."
2,table,awardsmanagers,awardsmanagers,3436,"CREATE TABLE ""awardsmanagers"" (\n\t""ID"" INTEGE..."
3,table,awardsplayers,awardsplayers,3445,"CREATE TABLE ""awardsplayers"" (\n\t""ID"" INTEGER..."
4,table,awardssharemanagers,awardssharemanagers,3603,"CREATE TABLE ""awardssharemanagers"" (\n\t""ID"" I..."


In [91]:
# data is only current to 2018! This is important as you draw
# conclusions from the data
pd.read_sql_query('select max(yearid) from halloffame', con) 

,max(yearid)
0,2018


## Deliverable 1

The database contains a table called "halloffame". Use a SELECT statement to print out the first five records of the halloffame table. Use a [limit clause](https://www.sqlitetutorial.net/sqlite-limit/) to only select the first five rows of the table. Note that you will need to use the pandas `read_sql_query` method which takes a connection arguments stored in the variable `con`. 

In [ ]:
my_sql_statement = None # your code here

pd.read_sql_query(my_sql_statement, con)

## Deliverable 2

What are the fields in the hall of fame table? What do you think they represent?

[Your answer here]

## Deliverable 3

The `halloffame` table includes a field called `playerid`. Often when you have an `id` field in a database it links to another table in the db. Often working with a database means using SQL to explore and learn the structure of the db. Use the `lahmans` dataframe from above to list the tables in the basebaseball database. Try to discover which table might contain a `playerid` field as a primary key. Then fill out the cell below to select the first five rows from the table. 

In [3]:
list_from_table = None # your query here
pd.read_sql_query(list_from_table, con)

## Deliverable 4

Looking again at the tables in the notebook, notice that there is an `allstarfull` table. Let's look at that table.

In [20]:
pd.read_sql_query('select * from allstarfull limit 5', con)

,ID,playerID,yearID,gameNum,gameID,teamID,team_ID,lgID,GP,startingPos
0,1,gomezle01,1933,0,ALS193307060,NYA,921,AL,1,1
1,2,ferreri01,1933,0,ALS193307060,BOS,912,AL,1,2
2,3,gehrilo01,1933,0,ALS193307060,NYA,921,AL,1,3
3,4,gehrich01,1933,0,ALS193307060,DET,919,AL,1,4
4,5,dykesji01,1933,0,ALS193307060,CHA,915,AL,1,5


Looking at the `halloffame` table and the `allstarfull` table we can start to form a question. 

Are there players that were often all stars but who were not elected to the hall of fame? Maybe these are **forgotten stars**! They should have made it to the hall of fame, but they were robbed! 

Notice that this question emerges from the process of exploring the data. This is a very common pattern.

To investigate the question, we need to find out how many all star games each player played in. This will require joins and aggregation. 

But before we are ready to proceed, we need to understand the data a little better. It is **very** common and **very** important to take time to understand data before drawing conclusions. In this case, it seems like the all_star table has a `gameNum` field.

What does that field represent? To investivate you will need to check the documentation or field list for the database in the [readme](http://www.seanlahman.com/files/database/readme2017.txt). Learning how to read and make sense of documentation (including data documentation) is an important skill. 

Another useful way to understand what information is represented in a database is to run SQL queries, such as the one below.

In [98]:
pd.read_sql_query('select distinct gameNum from allstarfull', con)

,gameNum
0,0
1,1
2,2


Based on your investigation, what does `gameNum` describe in the `allstarfull` table?

[Your answer here]

## Deliverable 5

- To start to answer our question, create dataframe called all_stars 
- To create the dataframe, use SQL to join allstarfull with the people table on playerId
- Select the following fields: 
    - 'allstarfull.playerID', 'nameLast', 'nameFirst', 'gameID'
    - 'allstarfull.playerID' specifies that you are taking the playerID field from the `allstarfull` table

In [ ]:
your_query = None
all_stars = pd.read_sql_query(your_query, con)
all_stars

## Deliverable 6

Pause to check and consider your results. The dataframe `all_stars` says the player aaronha01 played in all star games. Is that right? Well who is `aaronha01`? Use code to check your intuitions about this data by querying the `people` table to learn more about `aaronha01`. Answer by writing a select statement with a where clause.

In [ ]:
# delete me answer
your_query = None
rw = pd.read_sql_query(your_query, con)

## Deliverable 7 

Look up the player associated with the ID `aaronha01` on Wikipedia.  Who is aaronha01?

## Deliverable 8

Look at the next cell of code and describe what the three lines of code are doing. You may need to consult the pandas documentation.

- Line 1. Your explanation here
- Line 2. Your explanation here
- Line 3. Your explanation here

In [23]:
g = all_stars.groupby('playerID').size() # line 1
all_star_counts = g.reset_index() # line 2
all_star_counts = all_star_counts.rename(columns={0: "N_all_star_games"}) # line 3

## Deliverable 9

Sort `all_star_counts` on the field `N_all_star_games` (higher number of `N_all_star_games` should come first). Which players played in the most all star games? (You may need to join on the people table to get full names).

## Merging hall of fame data

Now let's look at data from the hall of fame table. Eventually, we will want to join the hall of fame table with the allstarfull table. But for now let's just take a look.

In [80]:
hall = pd.read_sql_query('select  * from halloffame', con)
hall

,ID,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
0,1,cobbty01,1936,BBWAA,226.0,170.0,222.0,Y,Player,None
1,2,ruthba01,1936,BBWAA,226.0,170.0,215.0,Y,Player,None
2,3,wagneho01,1936,BBWAA,226.0,170.0,215.0,Y,Player,None
3,4,mathech01,1936,BBWAA,226.0,170.0,205.0,Y,Player,None
4,5,johnswa01,1936,BBWAA,226.0,170.0,189.0,Y,Player,None
...,...,...,...,...,...,...,...,...,...,...
4186,4187,lidgebr01,2018,BBWAA,422.0,317.0,0.0,N,Player,None
4187,4188,millwke01,2018,BBWAA,422.0,317.0,0.0,N,Player,None
4188,4189,zambrca01,2018,BBWAA,422.0,317.0,0.0,N,Player,None
4189,4190,morrija02,2018,Veterans,NaN,NaN,NaN,Y,Player,None


## Deliverable 10

Is each player listed one time in the halloffame table? You will need to investigate the dataframe with pandas (Hint: use `value_counts`).

In [ ]:
# Your code and answer here

## Deliverable 11

How many times is the player `vanceda01` listed in the `halloffame` table? Why do you think that is? (Hint: look at the "inducted" field in the database documentation). Answer with code using the cell below. Note that a baseball player may be voted to be included in the hall of fame multiple times before they are accepted.

In [ ]:
# your code here

Adjust the `hall_sql_query` to include a where clause to only select rows from the `halloffame` table where inducted is "Y". Store your answer in the variable `inducted_query` below. The next cell should create a dataframe called `hall`. 

In [28]:
inducted_query = None
hall = pd.read_sql_query(inducted_query, con)

Now let's merge the `hall` dataframe with the `all_star_counts` dataframe. 

In [34]:

# I will use pandas to do the join. You could also do this in SQL. The API is similar. 
merged = pd.merge(all_star_counts, hall, how='left', on='playerID')
all_stars_vs_hall = merged[['playerID', "N_all_star_games", "inducted"]]
all_stars_vs_hall = all_stars_vs_hall.fillna(value="N")

## Deliverable 12
Sort the `all_stars_vs_hall` table by `N_all_star_games` in descending order.

In [ ]:
# your code here

## Deliverable 13

Make a table `missing_hall_of_famers` that shows the 10 players who played in the most all star games who were none the less NOT inducted into the hall of fame. You can filter the `all_stars_vs_hall` table for this, and call the `.head()` method on the filtered dataframe.

In [ ]:
missing_hall_of_famers = None

## Deliverable 14

Who are the players in your `missing_hall_of_famers` table? Try looking up a few of the players online (e.g. Wikipedia). Does your table make sense? Note: you may need to join the people table to see the players' full names. 

your answer here